In [3]:
from IPython.core.display import HTML
HTML("<style>.container { width:90% !important; }</style>")

In [47]:
import lsst.afw.image as afwImage
import lsst.obs.decam as obs_decam
import os
import numpy as np
import lsst.daf.persistence as dafPersist
import lsst.afw.display.ds9 as ds9
import lsst.afw.geom as afwGeom
import lsst.afw.detection as afwDetect
import lsst.afw.geom.ellipses as afwEllipse
import lsst.afw.cameraGeom as cameraGeom
import matplotlib.pyplot as plt
%matplotlib inline


import functions as fn
fn = reload(fn)

In [25]:
import pyfits as pf

filename = '/nfs/lsst2/photocalData/data/observer2/DECam_09000000.fits.fz'

this_file = pf.open(filename, mode='update')
primaryHDU = this_file[0]

primaryHDU.header['filename'] = 'DECam_09000000.fits'
primaryHDU.header['COADD_OF'] = 'DECam_00496489,DECam_00496492'
primaryHDU.header['COADNUMS'] = '496489,496492'
primaryHDU.header['expnum'] = 9000000

# this_file.close()


for key in primaryHDU.header:
    print key, primaryHDU.header[key]
    
# this_file.flush()
# this_file.close()


# for hdu in range(1, 70):
#     print 'hdu = %s'%hdu
#     out_file[hdu].data += this_file[hdu].data
# #             calexps[exp][ccdnum] = butler.get("calexp", dataId, ccdnum=ccdnum, immediate=True)
# #             print 'Fetched ' + calexps[exp][ccdnum].getDetector().getName(), calexps[exp][ccdnum].getDetector().getId(), 'for exp ' + str(exp)
# #         break

# out_file.flush()
# out_file.close()

    
# CoaddExposures([123,456,678])


#         for key in main_header:
#             print key, main_header[key]
#         exit()
#         for i in range(1,len(hdulist)):
#             print 'hdu %s'%i, 'CCDNUM = ', hdulist[i].header['CCDNUM'], 'Chip = ' , hdulist[i].header['DETPOS']


SIMPLE True
BITPIX 8
NAXIS 0
EXTEND True
NEXTEND 70
PROCTYPE RAW
PRODTYPE image
DETSIZE [1:29400,1:29050]
PIXSCAL1 0.27
PIXSCAL2 0.27
FILENAME DECam_09000000.fits
OBS-LONG 70.81489
TELESCOP CTIO 4.0-m telescope
OBSERVAT CTIO
OBS-LAT -30.16606
OBS-ELEV 2215.0
INSTRUME DECam
EXPREQ 10.0
EXPTIME 10.0
DARKTIME 11.09515
OBSID ct4m20151125t192142
DATE-OBS 2015-11-25T19:21:42.364995
TIME-OBS 19:21:42.364995
MJD-OBS 57351.8067403
OPENSHUT 2015-11-25T19:21:42.387180
RADESYS FK5
TIMESYS UTC
EXPNUM 9000000
OBJECT 63uA
OBSTYPE object
CAMSHUT Open
PROGRAM DECam Engineering
OBSERVER Abbott for Stubbs et al.
PROPOSER Walker
DTPI Walker
PROPID 2013A-9999
EXCLUDED HEXAPODGUIDER
AOS False
BCAM False
GUIDER 0
SKYSTAT False
FILTER r DECam SDSS c0002 6415.0 1480.0
FILTPOS cassette_4
INSTANCE DECam_20151125
ERRORS None
TELEQUIN 2000.0
TELSTAT Not tracking
RA 04:46:05.370
DEC -75:22:32.041
TELRA 22:15:30.938
TELDEC 30:28:35.998
HA -03:19:52.570
ZD 77.15
AZ 42.639
DOMEAZ 220.292
TELFOCUS UNKNOWN
VSUB True
GSK

In [33]:
a = this_file[20]
print a.size
print a.shape
print a.data.shape

8045387
(4146, 2160)
(4146, 2160)


In [39]:
fn.printHeaderData('/nfs/lsst2/photocalData/data/observer2/DECam_09000000.fits.fz')

SIMPLE True
BITPIX 8
NAXIS 0
EXTEND True
NEXTEND 70
PROCTYPE RAW
PRODTYPE image
DETSIZE [1:29400,1:29050]
PIXSCAL1 0.27
PIXSCAL2 0.27
FILENAME DECam_09000000.fits
OBS-LONG 70.81489
TELESCOP CTIO 4.0-m telescope
OBSERVAT CTIO
OBS-LAT -30.16606
OBS-ELEV 2215.0
INSTRUME DECam
EXPREQ 20.0
EXPTIME 20.0
DARKTIME 22.1779199
OBSID ct4m20151125t192142
DATE-OBS 2015-11-25T19:21:42.364995
TIME-OBS 19:21:42.364995
MJD-OBS 57351.8067403
OPENSHUT 2015-11-25T19:21:42.387180
RADESYS FK5
TIMESYS UTC
EXPNUM 9000000
OBJECT 63uA
OBSTYPE object
CAMSHUT Open
PROGRAM DECam Engineering
OBSERVER Abbott for Stubbs et al.
PROPOSER Walker
DTPI Walker
PROPID 2013A-9999
EXCLUDED HEXAPODGUIDER
AOS False
BCAM False
GUIDER 0
SKYSTAT False
FILTER r DECam SDSS c0002 6415.0 1480.0
FILTPOS cassette_4
INSTANCE DECam_20151125
ERRORS None
TELEQUIN 2000.0
TELSTAT Not tracking
RA 04:46:05.370
DEC -75:22:32.041
TELRA 22:15:30.938
TELDEC 30:28:35.998
HA -03:19:52.570
ZD 77.15
AZ 42.639
DOMEAZ 220.292
TELFOCUS UNKNOWN
VSUB True
G

In [55]:
from lsst.afw.image import ExposureF
from lsst.meas.algorithms.installGaussianPsf import InstallGaussianPsfTask, FwhmPerSigma

exposure = ExposureF(100, 100)
task = InstallGaussianPsfTask()

# run with default size
task.config.fwhm = 1.5 * FwhmPerSigma # default - is fine
# task.config.fwhm = 1.8 * FwhmPerSigma # largest you can set to without assert failing
# task.config.fwhm = 1.9 * FwhmPerSigma # 1.9 and up all fail with default size

# change the size and larger spots are fine
# task.config.width = 55 # setting the size to greater than 11 (the default) allows much larger PSFs
# task.config.fwhm =  8 * FwhmPerSigma # this is fine

# going to smaller PSFs is weird
# task.config.fwhm = 1.1 * FwhmPerSigma # smallest you can go without assert failing
# task.config.fwhm = 1.1 * FwhmPerSigma # this is not OK
# task.config.fwhm = 0.9 * FwhmPerSigma # nor is this

# task.config.width = 55 # changing the size up or down doesn't fix this (as you'd expect)
# task.config.fwhm = 0.9 * FwhmPerSigma # 


task.run(exposure=exposure)
measFwhm = exposure.getPsf().computeShape().getDeterminantRadius() * FwhmPerSigma
assert abs(measFwhm - task.config.fwhm) < 1e-3

AssertionError: 

In [75]:
fn.GetExpNumsFromDir('/nfs/lsst2/photocalData/data/calib_raw_NOAO/bias/')

Found 20 files in /nfs/lsst2/photocalData/data/calib_raw_NOAO/bias/
Opening...
496469^496470^496473^496474^496487^496488^496490^496491^496493^496494^496496^496497^496516^496517^496623^496624^496721^496722^496724^496725


([496469,
  496470,
  496473,
  496474,
  496487,
  496488,
  496490,
  496491,
  496493,
  496494,
  496496,
  496497,
  496516,
  496517,
  496623,
  496624,
  496721,
  496722,
  496724,
  496725],
 '496469^496470^496473^496474^496487^496488^496490^496491^496493^496494^496496^496497^496516^496517^496623^496624^496721^496722^496724^496725')

In [63]:
test = [496623, 496488, 496725, 496490, 496517, 496624, 496494, 496473, 496722, 496493, 496487, 496470, 496721, 496724, 496469, 496491, 496497, 496496, 496474, 496516]

test.sort()
print str(test).replace(', ','^')


[496469^496470^496473^496474^496487^496488^496490^496491^496493^496494^496496^496497^496516^496517^496623^496624^496721^496722^496724^496725]


In [107]:
def AddVisitsToFiles(path_list, startnum=0):
    import pyfits as pf
    import os,sys
    
    mapping = dict()
    mapping2 = dict()
    
    mapping2['DARK'] = ''
    mapping2['FLAT'] = ''
    mapping2['BIAS'] = ''

    for path in path_list:
        filenames = os.listdir(path)
        expNums = []
        for filename in filenames:
            if filename[0]=='.': continue
            this_file = pf.open(os.path.join(path,filename), mode='update')
            this_file[0].header['VISIT'] = startnum
            this_file[0].header['EXPNUM'] = startnum
            this_file[0].header['FILTER'] = 'no_filter'
            this_file[0].header['IMGTYPE'] = this_file[0].header['IMGTYTPE'] #fix typo & duplication problem

            mapping[os.path.join(path,filename)] = startnum
            mapping2[str(this_file[0].header['IMGTYPE'])] = mapping2[str(this_file[0].header['IMGTYPE'])] + str(startnum) + '^'
            startnum += 1
            this_file.flush()
            this_file.close()
            
    print 'Added visit ids to %s files'%startnum
    for key in sorted(mapping):
        print key, mapping[key]
    print mapping2

    
AddVisitsToFiles(['/nfs/lsst2/photocalData/data/monocam/calib/dark/','/nfs/lsst2/photocalData/data/monocam/calib/flat/'])

Added visit ids to 185 files
/nfs/lsst2/photocalData/data/monocam/calib/dark/113-03_dark_bias_002_20140709225639.fits 6
/nfs/lsst2/photocalData/data/monocam/calib/dark/113-03_dark_bias_003_20140709225654.fits 17
/nfs/lsst2/photocalData/data/monocam/calib/dark/113-03_dark_bias_006_20140709225739.fits 8
/nfs/lsst2/photocalData/data/monocam/calib/dark/113-03_dark_bias_008_20140709225809.fits 9
/nfs/lsst2/photocalData/data/monocam/calib/dark/113-03_dark_bias_012_20140709225909.fits 10
/nfs/lsst2/photocalData/data/monocam/calib/dark/113-03_dark_bias_013_20140709225924.fits 7
/nfs/lsst2/photocalData/data/monocam/calib/dark/113-03_dark_bias_015_20140709225954.fits 4
/nfs/lsst2/photocalData/data/monocam/calib/dark/113-03_dark_bias_016_20140709230009.fits 5
/nfs/lsst2/photocalData/data/monocam/calib/dark/113-03_dark_bias_017_20140709230025.fits 16
/nfs/lsst2/photocalData/data/monocam/calib/dark/113-03_dark_bias_018_20140709230040.fits 2
/nfs/lsst2/photocalData/data/monocam/calib/dark/113-03_dar

In [44]:
# fn.printHeaderData('/nfs/lsst2/photocalData/data/calib_repo_1/calib1/BIAS/2015-11-25/BIAS-2015-11-25-01.fits')
# fn.printHeaderData('/nfs/lsst2/photocalData/data/monocam/raw/2016_05_05/g16d125.001')
fn.printHeaderData('/nfs/lsst2/photocalData/data/monocam/raw/2016_05_05/g16d126.000')

#DATE-OBS 2016-05-05T12:40:39.693

SIMPLE True
BITPIX 16
NAXIS 2
NAXIS1 100
NAXIS2 100
DATE 2016-05-04T02:35:29
ORIGIN USNO-FS
TIMESYS UTC
DATE-OBS 2016-05-05
TELESCOP NO61
INSTRUME shutter
OBSERVER jps
OBJECT BIAS
IMAGETYP OBJECT
FILTER SDSSG
RA 09:57:09.42
DEC -09:25:48.7
EQUINOX 2000.0
RAENC 09:57:57.95
DECENC -09:29:59.7
UTC 12:43:30.087
ST 09:57:58.866
EXPTIME 10.0004
DARKTIME 10.0004
AIRMASS 1.405
GAIN 5
RDNOISE 5
DATASEC [6:100,1:100]
CCDSEC [1:95,1:100]
BIASSEC [1:5,1:100]
NSUM 1 1
LTM1_1 1
LTM1_2 0
LTM2_1 0
LTM2_2 1
LTV1 5
LTV2 0
DTM1_1 1
DTM1_2 0
DTM2_1 0
DTM2_2 1
DTV1 0
DTV2 0
SCALE 0.3252
PRESKIP 0
CCDIDENT 23
CCDFLIP 0
CCDXBIN 1
CCDYBIN 1
CCDGAIN 0
EXEC_VER ???
TEMPS 1 12.90 0.00 0.00 0.00 0.00 0.00 0.00 0.00
COMMENT VU01 UT=02:35:15.99  LST=09:57:57.95  HA=E0:00:00.00  EP=2016.340
VU02 2016d125
VU03         (ZeroPoint) RA=W0:00:00.00   Dec=+00:00:00.0
VU04         (2000.000)  RA=09:52:19.62   Dec=+35:11:04.3
VU05         (Apparent)  RA=09:53:17.62   Dec=+35:06:28.9
VU06         (Observed)  RA=09:53:17.70  

In [43]:
import pyfits as pf
import shutil
import sys

# fn.printHeaderData('/nfs/lsst2/photocalData/data/monocam/raw/2016_05_05/g16d125.001')
# fn.printHeaderData('/nfs/lsst2/photocalData/data/monocam/raw/2016_05_05/morningbias_02.fits')

path = '/nfs/lsst2/photocalData/data/monocam/raw/2016_05_05/'
files = os.listdir(path)



for i, filename in enumerate(files):
    if filename.find('g16')!=-1:continue
    if filename.find('USNO_partner')!=-1:continue

    bias_filename = path + filename
    bias_file = pf.open(bias_filename)
    splice_date = bias_file[0].header['DATE-OBS']
    print splice_date

    partner_filename = '/nfs/lsst2/photocalData/data/monocam/raw/2016_05_05/g16d126.' + str(i).zfill(3)
    shutil.copyfile('/nfs/lsst2/photocalData/data/monocam/raw/2016_05_05/g16d125.001', partner_filename)
    out_file = pf.open(partner_filename, mode='update')

    primaryHDU = out_file[0].header
    
    
    date, time = splice_date.split('T')

    
#     primaryHDU['DATE'] = splice_date
    primaryHDU['DATE-OBS'] = date
    primaryHDU['UTC'] = time

    primaryHDU['OBJECT'] = 'BIAS'
#     primaryHDU['OBJECT'] = bias_file[0].header['DATE-OBS']
#     primaryHDU['OBJECT'] = bias_file[0].header['DATE-OBS']
#     primaryHDU['OBJECT'] = bias_file[0].header['DATE-OBS']
#     primaryHDU['OBJECT'] = bias_file[0].header['DATE-OBS']



    out_file.flush()
    out_file.close()
    bias_file.close()


2016-05-05T12:43:30.087
2016-05-05T12:44:09.441
2016-05-05T12:41:23.256
2016-05-05T12:43:34.454
2016-05-05T12:43:51.942
2016-05-05T12:41:32.052
2016-05-05T12:40:39.693
2016-05-05T12:42:06.994
2016-05-05T12:43:38.829
2016-05-05T12:44:05.068
2016-05-05T12:43:21.336
2016-05-05T12:43:43.237
2016-05-05T12:40:57.031
2016-05-05T12:41:49.514
2016-05-05T12:42:37.639
2016-05-05T12:41:58.249
2016-05-05T12:41:05.772
2016-05-05T12:43:16.963
2016-05-05T12:41:36.380
2016-05-05T12:43:56.317
2016-05-05T12:42:59.471
2016-05-05T12:42:46.347
2016-05-05T12:41:40.757
2016-05-05T12:42:33.232
2016-05-05T12:42:55.139
2016-05-05T12:41:45.134
2016-05-05T12:41:14.518
2016-05-05T12:40:48.273
2016-05-05T12:42:20.120
2016-05-05T12:42:50.762
2016-05-05T12:43:03.841
2016-05-05T12:42:42.003
2016-05-05T12:40:52.652
2016-05-05T12:41:27.632
2016-05-05T12:41:10.177
2016-05-05T12:42:28.855
2016-05-05T12:41:53.872
2016-05-05T12:42:15.744
2016-05-05T12:42:02.622
2016-05-05T12:43:12.594
2016-05-05T12:42:24.494
2016-05-05T12:43

In [78]:
reload(fn)
data = fn.printHeaderData('/nfs/lsst2/photocalData/data/monocam/raw/2016_05_04/lsst1532+13_scienceII_08.fits', return_not_print=True)
for line in data[0:25]:
    print line
    
print '--------'
data = fn.printHeaderData('/nfs/lsst2/photocalData/data/monocam/raw/2016_05_04/g16d125.074', return_not_print=True)
for line in data[0:25]:
    print line

SIMPLE:True
BITPIX:16
NAXIS:0
EXTEND:True
HISTORY:Created with RTS2 version 0.9.4 build on Apr 12 2013 14:04:47.
CTIME:1462349801
USEC:104636
JD:2457512.84492
DATE-OBS:2016-05-04T08:16:41.104
EXPOSURE:300.0
EXPTIME:300.0
INSTRUME:112-04
TELESCOP:LSST lab
ORIGIN:BNL
SCRIPREP:0
SCRIPT:for 20 { E 300 }
SCR_COMM:
COMM_NUM:0
CCD_TYPE:
CCD_SER:
BINNING:1x1
BINX:1
BINY:1
SHUTTER:0
DATA_CHANNELS:16
--------
SIMPLE:True
BITPIX:16
NAXIS:2
NAXIS1:100
NAXIS2:100
DATE:2016-05-04T08:16:41
ORIGIN:USNO-FS
TIMESYS:UTC
DATE-OBS:2016-05-04
TELESCOP:NO61
INSTRUME:shutter
OBSERVER:jps
OBJECT:lsst1532+1
IMAGETYP:OBJECT
FILTER:SDSSI
RA:15:32:11.07
DEC:+13:56:38.0
EQUINOX:2000.0
RAENC:15:32:58.06
DECENC:+13:53:40.7
UTC:08:11:38.462
ST:15:35:15.922
EXPTIME:300.0004
DARKTIME:300.0004
AIRMASS:1.073


In [80]:
data = fn.printHeaderData('/nfs/lsst2/photocalData/data/monocam/raw/2016_05_04/lsst1532+13_02.fits', return_not_print=True)
for line in data[0:30]:
    print line
    
print '--------'
data = fn.printHeaderData('/nfs/lsst2/photocalData/data/monocam/raw/2016_05_04/g16d125.051', return_not_print=True)
for line in data[0:10]:
    print line

SIMPLE:True
BITPIX:16
NAXIS:0
EXTEND:True
HISTORY:Created with RTS2 version 0.9.4 build on Apr 12 2013 14:04:47.
CTIME:1462341047
USEC:88709
JD:2457512.7436
DATE-OBS:2016-05-04T05:50:47.088
EXPOSURE:30.0
EXPTIME:30.0
INSTRUME:112-04
TELESCOP:LSST lab
ORIGIN:BNL
SCRIPREP:0
SCRIPT:E 30
SCR_COMM:
COMM_NUM:0
CCD_TYPE:
CCD_SER:
BINNING:1x1
BINX:1
BINY:1
SHUTTER:0
DATA_CHANNELS:16
CHAN:16
CHAN1:True
CHAN2:True
CHAN3:True
CHAN4:True
--------
SIMPLE:True
BITPIX:16
NAXIS:2
NAXIS1:100
NAXIS2:100
DATE:2016-05-04T05:51:22
ORIGIN:USNO-FS
TIMESYS:UTC
DATE-OBS:2016-05-04
TELESCOP:NO61


In [82]:

data = fn.printHeaderData('/nfs/lsst2/photocalData/data/monocam/sync/20160510/BD+023375_i_02.fits', return_not_print=True)
for line in data[0:15]:
    print line
    
print '--------'
data = fn.printHeaderData('/nfs/lsst2/photocalData/data/monocam/sync/2016_05_11/g16d132.237', return_not_print=True)
for line in data[0:15]:
    print line

SIMPLE:True
BITPIX:16
NAXIS:0
EXTEND:True
HISTORY:Created with RTS2 version 0.9.4 build on May  6 2016 18:01:35.
CTIME:1462960731
USEC:518302
JD:2457519.91587
DATE-OBS:2016-05-11T09:58:51.518
EXPOSURE:5.0
EXPTIME:5.0
INSTRUME:113-03
TELESCOP:USNO 1.3m
ORIGIN:BNL
NUMBER:237
--------
SIMPLE:True
BITPIX:16
NAXIS:2
NAXIS1:100
NAXIS2:100
DATE:2016-05-11T09:58:59
ORIGIN:USNO-FS
TIMESYS:UTC
DATE-OBS:2016-05-11
TELESCOP:NO13
INSTRUME:ashutter
OBSERVER:mam
OBJECT:BD+023375
IMAGETYP:OBJECT
FILTER:SDSSI


In [79]:
data = fn.printHeaderData('/nfs/lsst2/photocalData/data/monocam/sync/2016_05_12/g16d133.001', return_not_print=True)
for line in data[0:25]:
    print line

SIMPLE:True
BITPIX:16
NAXIS:2
NAXIS1:100
NAXIS2:100
DATE:2016-05-12T02:35:16
ORIGIN:USNO-FS
TIMESYS:UTC
DATE-OBS:2016-05-12
TELESCOP:NO13
INSTRUME:ashutter
OBSERVER:mam
OBJECT:wakeup
IMAGETYP:OBJECT
FILTER:SDSSR
RA:11:07:38.65
DEC:+29:23:22.6
EQUINOX:2000.0
RAENC:11:08:31.26
DECENC:+29:18:11.8
ROTATOR:0.0
UTC:02:35:12.722
ST:10:29:27.373
EXPTIME:1.0008
DARKTIME:1.0008
